In [6]:
import redis
import pandas as pd
import numpy as np
import time

In [10]:
r = redis.Redis(host="localhost", port=6379, db=0)

In [5]:
r.json().set("test", "$", {"sim": 0.5}, nx=True)
print(r.json().get("test"))

{'sim': 0.5}


In [9]:
n = 4000
data = np.zeros((n, n), dtype=np.float16)

# Measure the time taken to populate the DataFrame
start_time = time.time()

for i in range(n):
    pipeline = r.pipeline()
    for j in range(n):
        pipeline.execute_command("JSON.GET", "test", "$.sim")

    results = pipeline.execute()

    data[i, :] = [float(result[0]) for result in results]

df = pd.DataFrame(data)
memory_usage = df.memory_usage(index=True).sum()
df = df.astype(np.float16)

end_time = time.time()
elapsed_time = end_time - start_time

# Verify the DataFrame
print(df.head())
print(f"Time taken to generate DataFrame: {elapsed_time:.2f} seconds")
print(f"Memory usage of DataFrame: {memory_usage / (1024 * 1024):.2f} MB")
del df

   0     1     2     3     4     5     6     7     8     9     ...  3990  \
0   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  ...   0.5   
1   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  ...   0.5   
2   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  ...   0.5   
3   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  ...   0.5   
4   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  ...   0.5   

   3991  3992  3993  3994  3995  3996  3997  3998  3999  
0   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  
1   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  
2   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  
3   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  
4   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5  

[5 rows x 4000 columns]
Time taken to generate DataFrame: 79.40 seconds
Memory usage of DataFrame: 30.52 MB
